In [ ]:
from Funciones import*


In [17]:
from keras.models import load_model
import numpy as np

import serial

flagADC = True

# Configura el puerto serial, ajusta el nombre del puerto según sea necesario
puerto_serial = serial.Serial(
    "COM4", 115200
)  # Reemplaza 'COM3' con el nombre de tu puerto


def read_binary_data(serial_port, data_size):
    data = serial_port.read(data_size)  # Lee los datos binarios
    return int.from_bytes(data, byteorder="little")  # Convierte a un entero


data_size = 2
end_of_transmission_char = 0x04

matriz_datos = []


datos_fila = []

    # Escuchar continuamente al puerto serial
while flagADC:
        # Intentar leer un dato
    try:
        linea_serial = read_binary_data(puerto_serial, data_size)
        if(linea_serial<7000):
            datos_fila.append(float(linea_serial))
    except Exception as e:
        print(f"Error al leer datos: {e}")
        break

        # Verificar si se ha recibido el carácter especial
    if puerto_serial.in_waiting > 0:
        if ord(puerto_serial.read()) == end_of_transmission_char:
            flagADC= False
            break


puerto_serial.close()
print(datos_fila)


muestras_test=[]

muestras_test = datos_fila[:-15]

matriz_np = np.array(muestras_test, dtype=float)

muestras_test = matriz_np.reshape(1, -1)



coefs_test = []

wavelet = 'coif6' 
nivel_descomposicion = 3  

for muestras_rango in muestras_test:
    coeficientes = transformadaWavelet(muestras_rango,wavelet,nivel_descomposicion)
    coefs_test.append(coeficientes)


recons_test = []

for coefs in coefs_test:
    senal_recons = reconstruccionWavelet(coefs,wavelet)
    recons_test.append(senal_recons)

datos_analizar = [recons_test]

caract_MAV = calcular_caracteristica_lista(datos_analizar, calcular_MAV)
caract_STD = calcular_caracteristica_lista(datos_analizar, calcular_STD)
caract_RMS = calcular_caracteristica_lista(datos_analizar, calcular_RMS)
caract_VAR = calcular_caracteristica_lista(datos_analizar, calcular_VAR)
caract_WL = calcular_caracteristica_lista(datos_analizar, calcular_WL)
caract_DMAV = calcular_caracteristica_lista(datos_analizar, calcular_DMAV)
caract_SSC = calcular_caracteristica_lista(datos_analizar, calcular_SSC)

X = [caract_MAV,caract_STD,caract_RMS,caract_VAR,caract_WL,caract_DMAV]

X = np.array(X).reshape(1, -1)

# from keras.models import load_model
# import numpy as np

# Cargar el modelo desde el archivo
loaded_model = load_model('NN Compiladas/modeloV2_V7.h5')

# Realizar predicciones en datos de prueba
predictions = loaded_model.predict(X)

# Obtener las etiquetas predichas como enteros
predicted_labels = np.argmax(predictions, axis=1)

# Mapear los enteros a los nombres de dedos
finger_mapping = {0: 'Dedo_Anular', 1: 'Dedo_Índice', 2: 'Dedo_Medio'}

# Convertir los enteros a nombres de dedos
predicted_fingers = [finger_mapping[label] for label in predicted_labels]

# Imprimir los dedos predichos
print("Dedos predichos:")
print(predicted_fingers)


[298.0, 122.0, 127.0, 123.0, 123.0, 160.0, 115.0, 181.0, 117.0, 157.0, 108.0, 238.0, 112.0, 114.0, 131.0, 158.0, 123.0, 339.0, 125.0, 116.0, 177.0, 107.0, 218.0, 129.0, 133.0, 118.0, 1108.0, 126.0, 119.0, 155.0, 138.0, 124.0, 119.0, 124.0, 117.0, 129.0, 130.0, 125.0, 117.0, 110.0, 113.0, 174.0, 109.0, 216.0, 121.0, 199.0, 128.0, 118.0, 109.0, 161.0, 130.0, 141.0, 124.0, 648.0, 111.0, 117.0, 119.0, 120.0, 132.0, 157.0, 118.0, 115.0, 136.0, 124.0, 127.0, 128.0, 155.0, 149.0, 119.0, 109.0, 112.0, 118.0, 106.0, 114.0]
1/1 [==============================] - 0s 86ms/step
Dedos predichos:
['Dedo_Medio']


In [ ]:
import pandas as pd
from datetime import datetime

matriz_medio = matriz_datos

df_medio_V2 = pd.DataFrame(matriz_medio)

df_medio_V2_sin_nan = df_medio_V2.fillna(0)
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
nombre_archivo = f"Matriz_Medio_V2_{timestamp}.csv"
df_medio_V2_sin_nan.to_csv(nombre_archivo,index=False)